In [1]:
res = {
    "class_instance": None
}

def class_decorator(cls):
   for name, method in cls.__dict__.iteritems():
        if hasattr(method, "use_class"):
            # do something with the method and class
            res["class_instance"] = cls
#             print(name, cls)
   return cls

def method_decorator(view):
    # mark the method as something that requires view's class
    view.use_class = True
    return view

@class_decorator
class ModelA(object):
    @method_decorator
    def a_method(self):
        # do some stuff
        print("a_method")
        pass
    
obj = ModelA()
print(res["class_instance"] == obj.__class__)
from pydoc import locate
a = locate("toto")
print(a)

True
None


In [3]:
from abc import ABCMeta, abstractmethod


class AbstractValidator(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def validate(self):
        pass

    @abstractmethod
    def fix(self):
        pass

    @abstractmethod
    @property
    def toto(self):
        pass

    
    
from typing import Any

from protocol0.validation.ValidatorInterface import AbstractValidator


class PropertyValidator(AbstractValidator):
    def __init__(self, obj, prop, expected_value):
        # type: (Any, str, Any) -> None
        self._object = obj
        self._property = prop
        self._expected_value = expected_value

    def validate(self):
        # type: () -> bool
        try:
            return getattr(self._object, self._property) == self._expected_value
        except AttributeError:
            return False

    def fix(self):
        # type: () -> None
        setattr(self._object, self._property, self._expected_value)

        
p = PropertyValidator(None, "toto", None)

AttributeError: 'property' object has no attribute '__isabstractmethod__'

In [17]:
a = ["toto", "titi"]
print("\n".join(a))

toto
titi
